In [111]:
import requests
import json
import pandas as pd
from datetime import date
import openpyxl
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [123]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')

In [165]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print("Data retrieved successfully:")
    print(data)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    print(f"Response: {response.text}")

Data retrieved successfully:
{'adverts': [{'type': 8, 'status': 9, 'count': 34, 'advert_list': [{'advertId': 16530753, 'changeTime': '2024-12-28T12:51:06.151959+03:00'}, {'advertId': 18447862, 'changeTime': '2025-01-22T11:54:53.488521+03:00'}, {'advertId': 18870595, 'changeTime': '2025-03-04T12:08:56.607322+03:00'}, {'advertId': 18870619, 'changeTime': '2025-01-27T17:57:22.639209+03:00'}, {'advertId': 18870701, 'changeTime': '2025-01-22T11:24:44.733987+03:00'}, {'advertId': 18870853, 'changeTime': '2025-01-22T11:32:52.171465+03:00'}, {'advertId': 19297608, 'changeTime': '2025-02-25T10:10:51.745756+03:00'}, {'advertId': 19297685, 'changeTime': '2025-02-25T10:05:05.170535+03:00'}, {'advertId': 20124734, 'changeTime': '2025-01-22T11:37:46.747581+03:00'}, {'advertId': 20139198, 'changeTime': '2024-12-14T15:09:30.141475+03:00'}, {'advertId': 20487381, 'changeTime': '2025-01-22T11:40:04.593972+03:00'}, {'advertId': 21202453, 'changeTime': '2024-12-28T15:21:22.362277+03:00'}, {'advertId': 212

In [169]:
# Flatten the JSON data
df = pd.json_normalize(
    data['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df['changeTime'] = pd.to_datetime(df['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
df = df[df['changeTime'].dt.year >= 2024]
# Reset the index and drop the old index
df = df.reset_index(drop=True)

# Display the updated DataFrame
df

,advertId,changeTime,type,status,count
0,16530753,2024-12-28 12:51:06.151959+03:00,8,9,34
1,18447862,2025-01-22 11:54:53.488521+03:00,8,9,34
2,18870595,2025-03-04 12:08:56.607322+03:00,8,9,34
3,18870619,2025-01-27 17:57:22.639209+03:00,8,9,34
4,18870701,2025-01-22 11:24:44.733987+03:00,8,9,34
...,...,...,...,...,...
135,21386314,2024-12-05 11:24:04.914806+03:00,9,7,16
136,21820825,2024-12-06 09:49:30.562364+03:00,9,7,16
137,22123202,2024-12-23 12:10:37.976556+03:00,9,7,16
138,11296434,2024-03-22 10:24:36.875685+03:00,5,7,118


In [8]:
filename = "Promotion.xlsx"
df.to_excel(filename, index=False)

In [7]:
unique_change_times = df['changeTime'].unique()
print(unique_change_times)

['2025-02-25T09:56:44.373405+03:00' '2025-02-25T09:56:36.977327+03:00'
 '2025-02-10T15:50:50.654674+03:00' '2025-02-10T15:50:46.711076+03:00'
 '2024-12-28T14:56:51.438476+03:00' '2025-02-10T15:50:45.190525+03:00'
 '2025-02-06T14:45:40.972398+03:00' '2024-12-28T14:59:03.412003+03:00'
 '2025-02-10T15:50:43.275656+03:00' '2025-02-25T10:07:05.519407+03:00'
 '2025-01-06T19:56:55.030411+03:00' '2025-01-31T16:14:36.03518+03:00'
 '2025-02-26T15:24:26.898332+03:00' '2025-01-24T17:28:07.565413+03:00'
 '2025-01-24T17:27:45.032317+03:00' '2025-03-03T10:51:20.471041+03:00'
 '2023-06-09T17:13:45.296789+03:00' '2023-07-17T17:56:44.162491+03:00'
 '2023-09-11T19:47:52.142401+03:00' '2023-09-13T19:02:34.535107+03:00'
 '2023-09-11T17:14:54.50553+03:00' '2023-09-13T14:53:09.428335+03:00'
 '2023-12-11T23:45:14.286592+03:00' '2023-12-11T22:31:06.941346+03:00'
 '2024-07-15T12:16:26.499587+03:00' '2024-04-11T10:53:48.400039+03:00'
 '2024-05-13T10:13:17.846636+03:00' '2024-05-20T16:18:30.678177+03:00'
 '2024-0

In [176]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks = [df['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df), chunk_size)]

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "change",  # Order by the "change" field
}

# Define the headers (replace 'KeyGuten' with your actual API token)
headers = {
    "Authorization": KeyGuten,  # Replace with your API token
    "Content-Type": "application/json"
}

# List to store all campaign data
all_campaign_data = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Send the POST request
    response = requests.post(url, params=query_params, json=chunk, headers=headers)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(1)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data.extend(data)
        print(f"Response for Chunk {idx + 1}: {data}")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")
        
        
campaigns_df = pd.DataFrame(all_campaign_data)
print("Combined Campaign Data:")
campaigns_df

Response for Chunk 1: [{'endTime': '2100-01-01T00:00:00+03:00', 'createTime': '2024-11-21T18:26:22.860916+03:00', 'changeTime': '2025-03-04T13:07:05.717861287+03:00', 'startTime': '2025-02-07T17:51:56.390438+03:00', 'autoParams': {'cpm': 580, 'subject': {'name': 'Ингаляторы', 'id': 665}, 'sets': [{'name': 'для всех', 'id': 2149}], 'MenuIds': None, 'nms': [171562370, 171562375], 'active': {'carousel': True, 'recom': False, 'booster': True}, 'nmCPM': [{'nm': 171562375, 'cpm': 680}, {'nm': 171562370, 'cpm': 680}]}, 'name': 'AND_Ингаляторы детские_2 SKU_21.11.2024', 'dailyBudget': 0, 'advertId': 21554118, 'status': 9, 'type': 8, 'paymentType': 'cpm'}, {'endTime': '2100-01-01T00:00:00+03:00', 'createTime': '2024-07-25T17:43:41.32226+03:00', 'changeTime': '2025-03-04T12:08:56.607322+03:00', 'startTime': '2025-01-09T16:45:28.974641+03:00', 'autoParams': {'cpm': 125, 'subject': {'name': 'Упаковщики вакуумные', 'id': 722}, 'sets': [{'name': 'для всех', 'id': 279}], 'MenuIds': None, 'nms': [2608

,endTime,createTime,changeTime,startTime,autoParams,name,dailyBudget,advertId,status,type,paymentType,searchPluseState,unitedParams,auction_multibids,params
0,2100-01-01T00:00:00+03:00,2024-11-21T18:26:22.860916+03:00,2025-03-04T13:07:05.717861287+03:00,2025-02-07T17:51:56.390438+03:00,"{'cpm': 580, 'subject': {'name': 'Ингаляторы',...",AND_Ингаляторы детские_2 SKU_21.11.2024,0,21554118,9,8,cpm,NaN,NaN,NaN,NaN
1,2100-01-01T00:00:00+03:00,2024-07-25T17:43:41.32226+03:00,2025-03-04T12:08:56.607322+03:00,2025-01-09T16:45:28.974641+03:00,"{'cpm': 125, 'subject': {'name': 'Упаковщики в...",CASO_Вакуумные упаковщики_2 SKU_25.07.24,0,18870595,9,8,cpm,NaN,NaN,NaN,NaN
2,2100-01-01T00:00:00+03:00,2024-11-21T18:25:03.219083+03:00,2025-03-04T09:50:05.126964325+03:00,2025-02-19T17:45:17.716489+03:00,"{'cpm': 610, 'subject': {'name': 'Ингаляторы',...",AND_Ингалятор CN-235_21.11.2024,0,21554091,9,8,cpm,NaN,NaN,NaN,NaN
3,2100-01-01T00:00:00+03:00,2024-12-23T13:18:53.855201+03:00,2025-03-04T00:43:04.038005+03:00,2025-03-04T00:43:04.038005+03:00,"{'cpm': 125, 'subject': {'name': 'Бритвы элект...",Metz_23.12.2024_АВТО,0,22255122,9,8,cpm,NaN,NaN,NaN,NaN
4,2100-01-01T00:00:00+03:00,2024-12-10T17:18:59.653075+03:00,2025-03-04T00:42:53.244027+03:00,2025-03-04T00:42:53.244027+03:00,NaN,AND_Массажер MH-102_10.12.2024_ВЧ,0,21965547,9,9,cpm,False,"[{'catalogCPM': 0, 'searchCPM': 150, 'subject'...","[{'nm': 159488075, 'bid': 790}]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2024-09-24T11:47:11.60881+03:00,2024-09-13T13:41:17.095187+03:00,2024-09-24T11:47:11.60881+03:00,2024-09-13T13:41:17.095187+03:00,"{'cpm': 100, 'subject': {'name': 'Кассеты для ...",VENUS_Кассеты_1 SKU_13.09.2024,0,19904919,7,8,,NaN,NaN,NaN,NaN
136,2024-09-04T20:53:36.970755+03:00,2024-08-30T17:30:27.468521+03:00,2024-09-04T20:53:36.970755+03:00,2024-08-30T17:30:27.468521+03:00,"{'cpm': 100, 'subject': {'name': 'Дезодоранты'...",OLD SPICE_Дезодоранты_30.08.2024,0,19626785,7,8,,NaN,NaN,NaN,NaN
137,2024-05-29T09:18:49.788779+03:00,2024-04-05T19:27:17.701332+03:00,2024-05-29T09:18:49.788779+03:00,2024-04-05T19:30:02.746413+03:00,NaN,Триммер Braun BT3321 05.04.2024,0,16144421,7,9,,False,"[{'catalogCPM': 0, 'searchCPM': 163, 'subject'...",NaN,NaN
138,2024-03-22T10:24:36.875685+03:00,2023-10-31T20:45:56.179187+03:00,2024-03-22T10:24:36.875685+03:00,2023-12-01T17:11:39.089065+03:00,NaN,Бритвенные станки Gillette Ассортимент P&G,0,11296434,7,5,,NaN,NaN,NaN,"[{'setName': 'Бритвы безопасные', 'intervals':..."


In [208]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "name", "advertId", "status", "type"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = campaigns_df[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

KeyError: "['endTime', 'createTime', 'name', 'status', 'type'] not in index"

In [117]:
# Flatten the JSON response into a DataFrame
df = pd.json_normalize(data, sep='_')

# Display the DataFrame
df

,adverts,all
0,"[{'type': 9, 'status': 8, 'count': 2, 'advert_...",417


In [172]:
import requests

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "change",  # Order by the "change" field

}

# Define the request body (list of campaign IDs)
# Replace with actual campaign IDs (maximum 50 IDs)
request_body = [20955474]

# Define the headers (replace 'your_api_token' with your actual API token)
headers = {
    "Authorization": KeyGuten,  # Replace with your API token
    "Content-Type": "application/json"
}

# Make the POST request
response = requests.post(url, params=query_params, json=request_body, headers=headers)

# Check the response status
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print("Response Data:", data)
else:
    print(f"Error: {response.status_code}, {response.text}")

Response Data: [{'endTime': '2024-10-31T19:09:55.666795+03:00', 'createTime': '2024-10-28T18:49:38.583434+03:00', 'changeTime': '2024-10-31T19:09:55.666795+03:00', 'startTime': '2024-10-31T12:15:15.669999+03:00', 'searchPluseState': False, 'name': 'SMEG_Блендер стац_28.10.2024', 'unitedParams': [{'catalogCPM': 0, 'searchCPM': 150, 'subject': {'id': 614, 'name': 'блендеры'}, 'MenuIds': None, 'menus': [{'id': 0, 'name': ''}], 'nms': [202777102, 149739854, 153936020, 208929076, 174090162]}], 'dailyBudget': 0, 'advertId': 20955474, 'status': 7, 'type': 9, 'paymentType': ''}]


In [29]:
# Flatten the 'unitedParams' field
df_param = pd.json_normalize(
    data,
    record_path='unitedParams',  # Path to the nested list
    meta=[
        'endTime', 'createTime', 'changeTime', 'startTime', 'searchPluseState',
        'name', 'dailyBudget', 'advertId', 'status', 'type', 'paymentType'
    ],  # Include these fields from the parent level
    meta_prefix='parent_',  # Add a prefix to meta fields to avoid conflicts
    sep='_'
)
df_param

,catalogCPM,searchCPM,MenuIds,menus,nms,subject_id,subject_name,parent_endTime,parent_createTime,parent_changeTime,parent_startTime,parent_searchPluseState,parent_name,parent_dailyBudget,parent_advertId,parent_status,parent_type,parent_paymentType
0,0,150,None,"[{'id': 0, 'name': ''}]","[202777102, 149739854, 153936020, 208929076, 1...",614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,


In [33]:
# Explode the 'nms' field to create a row for each value in the list
df_exploded = df_param.explode('nms')
df_exploded

,catalogCPM,searchCPM,MenuIds,menus,nms,subject_id,subject_name,parent_endTime,parent_createTime,parent_changeTime,parent_startTime,parent_searchPluseState,parent_name,parent_dailyBudget,parent_advertId,parent_status,parent_type,parent_paymentType
0,0,150,None,"[{'id': 0, 'name': ''}]",202777102,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,
0,0,150,None,"[{'id': 0, 'name': ''}]",149739854,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,
0,0,150,None,"[{'id': 0, 'name': ''}]",153936020,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,
0,0,150,None,"[{'id': 0, 'name': ''}]",208929076,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,
0,0,150,None,"[{'id': 0, 'name': ''}]",174090162,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,


In [45]:
# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/upd"

# Define the headers (replace 'your_api_token' with your actual API token)
headers = {
    "Authorization": KeyGiper,  # Replace with your API token
    "Content-Type": "application/json"
}

# Define the query parameters
query_params = {
    "from": "2025-02-17",  # Start date (replace with your desired date)
    "to": "2025-02-23"     # End date (replace with your desired date)
}

# Make the GET request
response = requests.get(url, headers=headers, params=query_params)

# Check the response status
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Transform the data into a pandas DataFrame
    df = pd.DataFrame(data)
    
    # Display the DataFrame
    print("DataFrame:")
    print(df)
else:
    print(f"Error: {response.status_code}, {response.text}")

DataFrame:
                              updTime                                campName  \
0           2025-02-23T23:59:59+03:00  Braun_6 серия_Электробритвы_31.01.2025   
1           2025-02-23T23:59:59+03:00             AND_Тонометр мех_27.01.2025   
2           2025-02-23T23:59:59+03:00               CASO_Яйцеварки_27.01.2025   
3           2025-02-23T23:59:59+03:00              ORAL-B_Щетки io_31.05.2025   
4           2025-02-23T23:59:59+03:00      Braun_Электробритвы_30.01.2025_РКП   
..                                ...                                     ...   
244  2025-02-17T14:42:58.636602+03:00      ELIKOR_Вытяжки 50см_29.01.2025_РКП   
245  2025-02-17T14:21:05.370467+03:00              Gillette_Наборы_20.11.2024   
246   2025-02-17T13:53:45.32265+03:00              ORAL-B_Щетки io_31.05.2025   
247  2025-02-17T11:00:35.931673+03:00  Braun_9 серия_Электробритвы_31.01.2025   
248  2025-02-17T10:55:14.005103+03:00     AND_Тонометр на запястье_27.01.2025   

    paymentType 

In [46]:
df

,updTime,campName,paymentType,updNum,updSum,advertId,advertType,advertStatus
0,2025-02-23T23:59:59+03:00,Braun_6 серия_Электробритвы_31.01.2025,Баланс,225639476,184,22950974,9,11
1,2025-02-23T23:59:59+03:00,AND_Тонометр мех_27.01.2025,Баланс,225639476,545,22845877,8,9
2,2025-02-23T23:59:59+03:00,CASO_Яйцеварки_27.01.2025,Баланс,225639476,364,22844946,8,9
3,2025-02-23T23:59:59+03:00,ORAL-B_Щетки io_31.05.2025,Баланс,225639476,1008,22954264,8,9
4,2025-02-23T23:59:59+03:00,Braun_Электробритвы_30.01.2025_РКП,Баланс,225639476,851,22931827,8,9
...,...,...,...,...,...,...,...,...
244,2025-02-17T14:42:58.636602+03:00,ELIKOR_Вытяжки 50см_29.01.2025_РКП,Баланс,224582459,323,22902062,8,9
245,2025-02-17T14:21:05.370467+03:00,Gillette_Наборы_20.11.2024,Баланс,224582459,1500,21526230,8,9
246,2025-02-17T13:53:45.32265+03:00,ORAL-B_Щетки io_31.05.2025,Баланс,224582459,1500,22954264,8,9
247,2025-02-17T11:00:35.931673+03:00,Braun_9 серия_Электробритвы_31.01.2025,Баланс,224582459,203,22951156,9,9


In [48]:
filename = "Promotion.xlsx"
df.to_excel(filename, index=False)

In [49]:
unique_change_times = df['paymentType'].unique()
print(unique_change_times)

['Баланс' 'Кэшбэк']


In [206]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Request headers
headers = {
    "Authorization": KeyGuten,  # Replace with your API token
    "Content-Type": "application/json"
}

# Define the date interval
date_interval = {
    "begin": "2025-02-17",  # Replace with your start date
    "end": "2025-02-19"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100
campaign_chunks = [filtered_df['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df), chunk_size)]

# List to store all campaign data
all_campaign_data = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(70)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data.extend(data)
        print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaigns_df = pd.json_normalize(all_campaign_data)

# Save the combined data to a text file
with open("response_data.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data:")
print(campaigns_df)

Processing Chunk 1: [21554118, 18870595, 21554091, 22255122, 21965547, 21883272, 23230882, 21577713, 19276694, 19276746, 19297608, 19276556, 19297685, 21498159, 21883285, 23408417, 21965489, 21821064, 21624823, 18870619, 22846531, 22846516, 22846446, 21639902, 18447862, 20487381, 20124734, 18870853, 18870701, 22165389, 21202745, 21203170, 21204193, 21204224, 21497699, 21497844, 21498270, 22255058, 21821199, 21821164, 21212715, 21212396, 21203996, 21202453, 16530753, 21203007, 20139198, 21203075, 21204098, 21204285, 23231292, 21624850, 21577733, 20397145, 20397161, 20954456, 20955117, 21041194, 21041387, 21041248, 21578389, 21698516, 22192773, 21577749, 21041312, 21041143, 19263710, 19262099, 17266442, 18870731, 17563748, 18870828, 18640342, 17688635, 17563873, 17687135, 16255090, 17687357, 17563938, 17687293, 17688586, 16913380, 16923161, 16919369, 16913696, 17687241, 16918415, 16923933, 16912972, 16912359, 16913774, 16914258, 16917733, 16917791, 16920802, 16923906, 16924292, 16918232,

In [209]:
campaigns_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval.begin,interval.end
0,6193,184,2.97,5.33,979.93,17,5,2.72,5,5745,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 159121...",22846446,2025-02-17,2025-02-19
1,6959,261,3.75,4.84,1262.43,25,10,3.83,10,48278,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 462925...",18870619,2025-02-17,2025-02-19
2,20014,507,2.53,9.35,4742.67,15,0,0.00,0,0,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 286529...",21577733,2025-02-17,2025-02-19
3,5411,170,3.14,3.00,510.23,8,0,0.00,0,0,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 263725...",21202453,2025-02-17,2025-02-19
4,11668,368,3.15,7.29,2683.64,22,0,0.00,0,0,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...",NaN,21204224,2025-02-17,2025-02-19
5,1673,52,3.11,7.24,376.49,3,0,0.00,0,0,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...",NaN,21204285,2025-02-17,2025-02-19
6,11858,330,2.78,6.22,2054.07,16,1,0.30,1,6210,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 263764...",20397161,2025-02-17,2025-02-19
7,6711,330,4.92,4.17,1375.81,16,3,0.91,3,21007,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...",NaN,21202745,2025-02-17,2025-02-19
8,3509,151,4.30,3.72,561.44,11,2,1.32,2,9360,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...",NaN,21204098,2025-02-17,2025-02-19
9,4013,54,1.35,17.50,944.81,0,0,0.00,0,0,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 286307...",21577713,2025-02-17,2025-02-19


In [210]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Request headers
headers = {
    "Authorization": KeyGuten,
    "Content-Type": "application/json"
}

# Define the payload using the correct schema
payload = [
    {
        "id": 21554118,  # Replace with your campaign ID
           "interval": {
                "begin": "2025-02-17",
                "end": "2025-02-23"
            } # Array of dates
    }
]

# Send POST request
response = requests.post(url, headers=headers, json=payload)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    data = response.json()
    
        # Save the response data into a text file
    with open("response_data.txt", "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)  # Pretty-print JSON into the file
    
    df = pd.DataFrame(data)
    print(json.dumps(data, indent=4))  # Pretty-print the JSON response
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(f"Response: {response.text}")

Request successful!
[
    {
        "interval": {
            "begin": "2025-02-17",
            "end": "2025-02-23"
        },
        "views": 34078,
        "clicks": 1187,
        "ctr": 3.48,
        "cpc": 12.88,
        "sum": 15287.5,
        "atbs": 136,
        "orders": 27,
        "cr": 2.27,
        "shks": 27,
        "sum_price": 78812,
        "days": [
            {
                "date": "2025-02-17T03:00:00+03:00",
                "views": 4649,
                "clicks": 202,
                "ctr": 4.35,
                "cpc": 2.53,
                "sum": 510.72,
                "atbs": 29,
                "orders": 6,
                "cr": 2.97,
                "shks": 6,
                "sum_price": 17382,
                "apps": [
                    {
                        "views": 400,
                        "clicks": 7,
                        "ctr": 1.75,
                        "cpc": 6.12,
                        "sum": 42.83,
                        "at

In [211]:
df

,interval,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId
0,"{'begin': '2025-02-17', 'end': '2025-02-23'}",34078,1187,3.48,12.88,15287.5,136,27,2.27,27,78812,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 171562...",21554118


In [212]:
# Flatten the JSON data
flattened_data = []

for entry in data:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

# Create a DataFrame
df = pd.DataFrame(flattened_data)
# Convert the 'date' column to datetime
df["date"] = pd.to_datetime(df["date"])
# Remove timezone information (make it timezone-naive)
df["date"] = df["date"].dt.tz_localize(None)
# Display the DataFrame
df


,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-02-17 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,268,6,2.24,4.80,28.78,0,0,0.00,0,0,21554118
1,2025-02-17 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,132,1,0.76,14.05,14.05,0,0,0.00,0,0,21554118
2,2025-02-17 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,867,40,4.61,2.36,94.49,6,1,2.50,1,2833,21554118
3,2025-02-17 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,2230,108,4.84,2.20,237.81,15,3,2.78,3,8787,21554118
4,2025-02-17 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,861,41,4.76,2.42,99.38,6,1,2.44,1,2929,21554118
5,2025-02-17 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,291,6,2.06,6.04,36.21,2,1,16.67,1,2833,21554118
6,2025-02-18 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,190,4,2.11,20.08,80.32,1,0,0.00,0,0,21554118
7,2025-02-18 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,296,6,2.03,19.80,118.79,1,1,16.67,1,2929,21554118
8,2025-02-18 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,2314,107,4.62,8.76,937.55,14,5,4.67,5,14645,21554118
9,2025-02-18 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,1320,39,2.95,14.63,570.56,5,1,2.56,1,2833,21554118


In [213]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped = (
    df.groupby([df["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Rename the 'date' column to 'day' for clarity
df_grouped.rename(columns={"date": "day"}, inplace=True)

# Display the grouped DataFrame
df_grouped



,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,171562370,2025-02-17 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,1290,47,2.476667,7.483333,144.75,8,2,6.390000,2,5666,21554118
1,171562375,2025-02-17 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,3359,155,3.946667,3.140000,365.97,21,4,1.740000,4,11716,21554118
2,171562370,2025-02-18 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,2054,53,2.300000,19.660000,893.57,7,1,0.853333,1,2833,21554118
3,171562375,2025-02-18 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,3627,173,4.183333,11.860000,1477.36,21,7,7.670000,7,20503,21554118
4,171562370,2025-02-19 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,2842,74,2.040000,70.753333,1800.81,7,1,0.640000,1,2833,21554118
5,171562375,2025-02-19 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,4261,166,3.086667,32.116667,2527.41,21,4,1.943333,4,11716,21554118
6,171562370,2025-02-20 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,2481,63,2.466667,26.810000,1512.57,10,1,0.876667,1,2833,21554118
7,171562375,2025-02-20 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,1648,63,3.296667,22.030000,926.85,9,1,0.833333,1,3280,21554118
8,171562370,2025-02-21 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,1967,49,1.886667,43.086667,1050.94,6,2,1.803333,2,5666,21554118
9,171562375,2025-02-21 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,1406,46,2.746667,20.470000,582.36,2,0,0.000000,0,0,21554118


In [214]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped.merge(
    filtered_df[["advertId", "endTime", "createTime", "name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,endTime,createTime,name_campaign,status,type
0,171562370,2025-02-17 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,1290,47,144.75,8,2,2,5666,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания
1,171562375,2025-02-17 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,3359,155,365.97,21,4,4,11716,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания
2,171562370,2025-02-18 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,2054,53,893.57,7,1,1,2833,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания
3,171562375,2025-02-18 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,3627,173,1477.36,21,7,7,20503,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания
4,171562370,2025-02-19 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,2842,74,1800.81,7,1,1,2833,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания
5,171562375,2025-02-19 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,4261,166,2527.41,21,4,4,11716,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания
6,171562370,2025-02-20 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,2481,63,1512.57,10,1,1,2833,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания
7,171562375,2025-02-20 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,1648,63,926.85,9,1,1,3280,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания
8,171562370,2025-02-21 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,1967,49,1050.94,6,2,2,5666,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания
9,171562375,2025-02-21 03:00:00,Ингалятор небулайзер компрессорный детский CN-...,1406,46,582.36,2,0,0,0,21554118,2100-01-01,2024-11-21,AND_Ингаляторы детские_2 SKU_21.11.2024,Идут показы,Автоматическая кампания


In [189]:
filename = "Compaings.xlsx"
df_grouped.to_excel(filename, index=False)

In [68]:
# Initialize an empty list to store flattened data
flattened_data = []

# Loop through the nested structure to extract relevant data
for day in data[0]['days']:
    for app in day['apps']:
        app_type = app['appType']
        for nm in app['nm']:
            flattened_data.append({
                'nmId': nm['nmId'],
                'appType': app_type
            })

# Convert the flattened data into a DataFrame
df = pd.DataFrame(flattened_data)

# Group by nmId and calculate the sum of appType
result = df.groupby('nmId', as_index=False).agg({'appType': 'sum'})

# Display the resulting DataFrame
print(result)

        nmId  appType
0  135577637      679
1  137609254      679
2  137609323      679
3  137609342      679
4  137609383      485
5  137609392      679
6  137609742      291


In [ ]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks = [df['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df), chunk_size)]

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "change",  # Order by the "change" field
}

# Define the headers (replace 'your_api_token' with your actual API token)
headers = {
    "Authorization": KeyGuten,  # Replace with your API token
    "Content-Type": "application/json"
}

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Send the POST request
    response = requests.post(url, params=query_params, json=chunk, headers=headers)
    time.sleep(2)
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        print(f"Response for Chunk {idx + 1}: {data}")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

[[20397145, 20397161, 20954456, 20955117, 21041143, 21041194, 21041248, 21041312, 21041387, 21577733, 21577749, 21578389, 21624850, 21698516, 22192773, 23231292, 7202847, 8004530, 8743562, 9496209, 9496385, 9591336, 11296204, 11296333, 16255090, 16301721, 16301819, 16912359, 16912972, 16913380, 16913696, 16913774, 16914258, 16917733, 16917791, 16918232, 16918415, 16919369, 16920802, 16923161, 16923906, 16923933, 16924292, 17266442, 17563748, 17563873, 17563938, 17687135, 17687241, 17687293], [17687357, 17688586, 17688635, 18640342, 18870731, 18870828, 19262099, 19263710, 19263824, 19265149, 19265205, 19626785, 19662812, 19904919, 19994021, 20039654, 20139599, 20175678, 20334946, 20354793, 20397195, 21040251, 21040485, 21040635, 21040756, 21040856, 21040946, 21385613, 21386362, 21482495, 21482904, 21498202, 8618594, 8743328, 462902, 462934, 462959, 462991, 463002, 463125, 463138, 463196, 463211, 463364, 480144, 480165, 480178, 480275, 480291, 480347], [480356, 480455, 480513, 480527, 48

' # Define the API endpoint\nurl = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"\n\n# Define the query parameters\nquery_params = {\n    "status": 9,  # Example: campaigns with status "9 - идут показы"\n    "type": 8,    # Example: automatic campaigns\n    "order": "change",  # Order by the "change" field\n    "direction": "asc"  # Ascending order\n}\n\n# Define the headers (replace \'your_api_token\' with your actual API token)\nheaders = {\n    "Authorization": "Bearer your_api_token",  # Replace with your API token\n    "Content-Type": "application/json"\n}\n\n# Iterate over each chunk and send a POST request\nfor idx, chunk in enumerate(campaign_chunks):\n    print(f"Processing Chunk {idx + 1}: {chunk}")\n    \n    # Send the POST request\n    response = requests.post(url, params=query_params, json=chunk, headers=headers)\n    \n    # Check the response status\n    if response.status_code == 200:\n        # Parse the JSON response\n        data = response.json()\n   